In [44]:
import numpy as np
import warnings
import pandas as pd
import os

In [45]:
from load_contracts import read_contract, preprocess_text
from load_info import get_security_names, load_info_xlsx
from manage_triggers import get_proccessed_triggers

In [46]:
BEGINING_OF_IV_INTRO_TEXT = get_proccessed_triggers("Begining_IV")
END_OF_IV_INTRO_TEXT = get_proccessed_triggers("End_IV_intro")

In [94]:
filename = "contracts/135_ActelisNetworks_COI_01072005.pdf"
text = read_contract(filename)
text = text.split(" ")
text = list(filter(lambda a: a != '', text))

### Utils

In [71]:
def find_loc(text, triggers, allow_contains=True):
    for trigger in triggers:
        curIndex = find_target(text, trigger, allow_contains=allow_contains)
        if curIndex != []:
            if len(curIndex) > 1:
                pass
#                 print("target occurs at multiple indicies")
            return curIndex
#     print("No trigger found in text")
    return None

In [72]:
def get_filepath(target_filename):
    dropbox_path = "/Users/DanielLongo/Dropbox/VC RA Avinika Narayan/Contracts project/coi/Done OCR'd/"
    for root, dirs, files in os.walk(dropbox_path):
        for dir in dirs:
            for filename in os.listdir(dropbox_path + dir):
                if filename == target_filename:
                    return dropbox_path + dir + "/" + filename
#     print(target_filename, "not found")

In [73]:
get_filepath("17445_955Dreams_COI_10112013.pdf")

"/Users/DanielLongo/Dropbox/VC RA Avinika Narayan/Contracts project/coi/Done OCR'd/955 Dreams/17445_955Dreams_COI_10112013.pdf"

In [74]:
def remove_periods(text): #doesn't remove decimal places
    out = []
    for word in text:
        if is_num(word):
            out += [word]
            continue
        out += word.split('.')
    return out

In [75]:
# def get_closest 

In [76]:
def is_num(x):
    x = x.strip('$')
    try:
        float(x)
    except ValueError:
        return False
    return True

In [77]:
def revert_secuirty_names(df):
    df.loc[df["Security Name"] == "common stock", "Security Name"] = "common"
    return df

In [78]:
def get_closest_string(values, target, less, any=False):
    if not any:
        if less:
            values = [x for x in values if x < target]
        else:
            values = [x for x in values if x > target]
    try:
        val = min(values, key=lambda x: abs(target - x))
    except ValueError:
        return -1
    return val

In [79]:
def find_target(text, target, allow_contains=True):
    target_loc = 0
    indicies = []
    start_index = 0
    for i in range(len(text)):
        word = text[i]
#         print("target_loc", target_loc)
        if word == target[target_loc] or (allow_contains and word in target[target_loc]):
            if target_loc == 0: #marks start of sequence
                start_index = i
            target_loc += 1
            if target_loc == len(target):
                indicies += [start_index]
                target_loc = 0
        elif word == target[0] or (allow_contains and word in target[0]):
            target_loc = 1
        else:
            if target_loc > 0:
                #reset just incase ababcd 
                i -= (target_loc - 1) 
            target_loc = 0
    return indicies

### Search Functions

In [119]:
def get_closest_num(values, target, less, min=-1, max=1e10000, any=False):
    try:
        index = int(target)
    except ValueError:
        indicies = find_target(values, target.split(" "))
#         print("Indicies", indicies)
        if len(indicies) > 1:
            warnings.warn("Indicies length greath than one", RuntimeWarning)
            for i in range(len(indicies)):
                if less:
                    cur_target = indicies[i]
                else:
                    cur_target = indicies[len(indicies) - 1 -i] #starts at end of doc
                cur_val = get_closest_num(values, cur_target, less, min=min, max=max, any=any)
                if (cur_val != None):
                    return cur_val
            warnings.warn("No number found", RuntimeWarning)
            print("No num found target:", target, "indicies:", indicies)
            return None
#         assert(len(indicies) > 1), "Invalid target"
        index = indicies[-1]
    if any: #doesn't get closest
        iters = range(0, len(values))
    elif less:
        iters = range(index -1, -1, -1) #0 is last
    else:#less = false
        iters = range(index, len(values))
    for i in iters:
        if (is_num(values[i])):
            return float(values[i].strip("$"))
    print("No num", target, "index", index)
    return None
    

In [120]:
def get_names(text):
    unsplit_names = get_security_names()
    names = [name.split(" ") for name in unsplit_names]
    indicies = []
    types = []
    for name in names:
        if (len(name) == 1):
            name += ["stock"]
        cur_indicies = find_target(text, name)
        if len(cur_indicies) > 0:
            types += [" ".join(name)]
            indicies += [cur_indicies]
    #remove extra "prefered stock"
    has_extra_prefered = True
    if "prefer stock" in types:
        for type in types:
            if type[-6:] == 'prefer':
                has_extra_prefered = True
                break
        if (has_extra_prefered):
            indicies.pop(types.index('prefer stock'))
            types.remove('prefer stock')
    return indicies, types

In [121]:
def get_num_of_shares(text, names):
    out = []
    for name in names:
        out += [get_closest_num(text, name, less=True)]
    return out
        

In [122]:
def get_types(names):
    types = []
    for name in names:
        if "common" in name:
            types += ["common"]
        elif "prefer" in name:
            types += ["prefered"]
        else:
            print("name doesn't contain common or prefer:", "".join(name))
            types += ["No type found"]
    return types

In [123]:
def generate_stats(IV_intro_text):
    _, names = get_names(IV_intro_text)
    types = get_types(names)
    nums = get_num_of_shares(IV_intro_text, names)
    d = {"Security Name":names, "Security Type":types, "Number":nums}
    df = pd.DataFrame(d)
    df = revert_secuirty_names(df)
    return df

In [124]:
unsplit_names = get_security_names()

In [125]:
all_names = [name.split(" ") for name in unsplit_names]

In [126]:
indicies = []
for name in all_names:
    indicies += [find_target(text, name)]


In [127]:
index_of_article_IV = find_target(text, ["articl", "fourth"])

### Search Through IV Intro

In [128]:
end_of_IV_intro = find_target(text, END_OF_IV_INTRO_TEXT)

In [129]:
# assert(len(end_of_IV_intro) == 1)

In [130]:
# end_of_IV_intro = end_of_IV_intro[0]

In [131]:
# articl_forth_occur = find_target(text, ["articl", "fourth"])

In [132]:
# begining_of_IV = get_closest_string(values = articl_forth_occur, target=end_of_IV_intro, less=True)

In [102]:
# print("IV Intro starts", begining_of_IV, "IV Intro ends", end_of_IV_intro)

In [101]:
# " ".join(text[begining_of_IV:end_of_IV_intro])

In [105]:
# IV_intro_text = text[begining_of_IV:end_of_IV_intro]
IV_intro_text = get_IV_intro_text(filename, BEGINING_OF_IV_INTRO_TEXT, END_OF_IV_INTRO_TEXT)

begining 212 end 372


In [106]:
indicies, names = get_names(IV_intro_text)

In [107]:
types = get_types(names)

In [110]:
get_closest_num(IV_intro_text, names[0], False)

0.01

In [111]:
get_closest_num(IV_intro_text, names[0], False)

0.01

In [116]:
" ".join(IV_intro_text)

'fourth compani is author to issu two class of share design respect common stock and prefer stock the compani is author to issu 43000000 share of common stock with a par valu of $0.01 per share the compani is author to issu 26007500 share of prefer stock 800000 of which are design seri a prefer stock with a par valu of $0.01 per share seri a prefer 700000 of which are design seri b prefer stock with a par valu of $0.01 per share seri b prefer 1507500 of which are design seri c prefer stock with a par valu of $0.01 per share seri c prefer and 23000000 of which are design seri d prefer stock with a par valu of $0.01 per share seri d prefer for the purpos of thi amend and restat certif of incorpor the term prefer stock shall mean seri a prefer seri b prefer seri c prefer and seri d prefer collectively th'

In [133]:
nums = get_num_of_shares(IV_intro_text, names)

Indicies [12, 26]
No num 12 index 12
Indicies [113, 125, 155]
Indicies [52, 64, 145]
Indicies [72, 84, 148]
Indicies [92, 104, 151]
Indicies [113, 125, 155]
Indicies [114, 126, 156]
Indicies [52, 64, 145]
Indicies [113, 125, 155]
Indicies [72, 84, 148]
Indicies [52, 64, 145]
Indicies [52, 64, 72, 84, 145, 148]
Indicies [114, 126, 156]
Indicies [52, 64, 145]
Indicies [113]
Indicies [52, 64, 145]
Indicies [72, 84, 148]
Indicies [92, 104, 151]
Indicies [113, 125, 155]
Indicies [52, 64, 145]
Indicies [52, 64, 145]


In [113]:
print(names)
print(types)
print(nums)

['common stock', 'seri seed prefer', 'seri a prefer', 'seri b prefer', 'seri c prefer', 'seri d prefer', 'founder prefer', 'seri aa prefer', 'seri dprefer', 'seri bb prefer', 'seri aaa prefer', 'seri ab prefer', 'undesign prefer', 'seri 2a prefer', 'seri seed prefer stock', 'seri a1 prefer', 'seri b1 prefer', 'seri c1 prefer', 'seri d1 prefer', 'seri a2 prefer', 'seri a3 prefer']
['common', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered', 'prefered']


NameError: name 'nums' is not defined

In [134]:
generate_stats(IV_intro_text)

Indicies [12, 26]
No num 12 index 12
Indicies [113, 125, 155]
Indicies [52, 64, 145]
Indicies [72, 84, 148]
Indicies [92, 104, 151]
Indicies [113, 125, 155]
Indicies [114, 126, 156]
Indicies [52, 64, 145]
Indicies [113, 125, 155]
Indicies [72, 84, 148]
Indicies [52, 64, 145]
Indicies [52, 64, 72, 84, 145, 148]
Indicies [114, 126, 156]
Indicies [52, 64, 145]
Indicies [113]
Indicies [52, 64, 145]
Indicies [72, 84, 148]
Indicies [92, 104, 151]
Indicies [113, 125, 155]
Indicies [52, 64, 145]
Indicies [52, 64, 145]


,Security Name,Security Type,Number
0,common,common,43000000.0
1,seri seed prefer,prefered,23000000.0
2,seri a prefer,prefered,800000.0
3,seri b prefer,prefered,700000.0
4,seri c prefer,prefered,1507500.0
5,seri d prefer,prefered,23000000.0
6,founder prefer,prefered,23000000.0
7,seri aa prefer,prefered,800000.0
8,seri dprefer,prefered,23000000.0
9,seri bb prefer,prefered,700000.0


### Evalutate Search Methods

In [41]:
def get_IV_intro_text(filename, begining_intro_triggers, end_intro_triggers):
    text = read_contract(filename)
    text = text.split(" ")
#     print("text", text)
#     text = text.split(" ")
#     text = remove_periods(text)
        
    if filename in Failures:
        return "Failure expected"
    try:
        begining_of_IV_intro = find_loc(text, begining_intro_triggers, allow_contains=False)
        end_of_IV_intro = find_loc(text, end_intro_triggers)[0]
#         print("end IV sample", " ".join(text[end_of_IV_intro: end_of_IV_intro + 50])
#         end_of_IV_intro = end_of_IV_intro[0]
#         end_of_IV_intro = 
    except TypeError:
        print("failed", filename)
        print(text)
        return None
    assert(begining_of_IV_intro != None), "Invalid info file " + " ".join(text) + filename
        
#     assert(end_of_IV_intro != None), "END Sequence not found"
#     articl_forth_occur = find_target(text, ["articl", "fourth"])
#     assert(articl_forth_occur != None), "Article fourth Sequence not found"
    begining_of_IV = get_closest_string(values = begining_of_IV_intro, target=end_of_IV_intro, less=True)
    if begining_of_IV == -1:
        print("no begining found")
        begining_of_IV = end_of_IV_intro - 100
    print("begining", begining_of_IV, "end", end_of_IV_intro)
    IV_intro_text = text[begining_of_IV:end_of_IV_intro]
    return IV_intro_text

In [42]:
def test_methods(filename, n=-1, start_index=0):
    iters = 0
    y = load_info_xlsx("rs1 database AN.xlsx", np_array=False)
    prev_filename = ""
    s, f = 0, 0
    for filename in y["File Name"]:
        if (start_index > 0):
            start_index -= 1
            continue
        if filename == prev_filename:
            continue
        prev_filename = filename
        iters += 1
        if iters == n:
            break
#         print(filename)
        filepath = get_filepath(filename)
        if filepath == None:
            continue
            
        #Test load intro text   
        Intro_IV_text = get_IV_intro_text(filepath, BEGINING_OF_IV_INTRO_TEXT, END_OF_IV_INTRO_TEXT)
        if (Intro_IV_text != None):
            s += 1
        else:
            return
            f += 1
        print(" ".join(Intro_IV_text))
        #Test load stats
#         stats = generate_stats(Intro_IV_text)
#         print(stats)
            
    print("SUCCESS", s)
    print("FAILURE", f)
        
#     text = get_IV_intro_text(filename)
#     stats = generate_stats(text)
    
    

In [43]:
test_methods(filename, n=-1, start_index=0)

no begining found
begining 489 end 589
gener corpor law articl iv author shares th total number of share of all class o f stock which the corpor shall have author to issu is a twelv million 12000000 share of common stock $0.0001 par valu per share common stock and b two million eight hundr thousand 2800000 share o f prefer stock $0.0001 par valu per share prefer stock a of the effect date of thi restat certif of incorpor thi restat certif all two million eight hundr thousand 2800000 share o f the author prefer stock of the corpor are herebi design seri seed prefer stock
no begining found
begining 519 end 619
author to issu is a 15000000 share of common stock $0.0001 par valu per share common stock and b 7021906 share of prefer stock $0.0001 par valu per share prefer stock a of the effect date of thi third restat certif of incorpor thi restat certif 2717681 share of the author prefer stock of the corpor are herebi design seri seed prefer stock 1 414225 share of the author prefer stock o

begining 621 end 3072
iv shall not advers affect ani contract or other right to indemnif of ani agent of the corpor that exist at or prior to the time of such amend repeal or modification articl v1 author of share thi corpor is author to issu two 2 class of share design common stock and prefer stock respect both of which shall have no par valu the total number of share of common stock author to be issu is170000000 share the total number of share of prefer stock author to be issu is 116655843 share 35892856 of which are design seri a prefer stock 43262987 of which are design seri b prefer stock and 37500000 of which are design seri c prefer stock 1 varticl vith follow is a statement of the right prefer privileg and restrict grant to and impos on each class of capit stock of the corpor unless otherwis indic refer to section or subsect in thi articl refer to section and subsect of thi articl vi a common stock1 gener the vote dividend and liquid right of the holder o f common stock are su

begining 14648 end 15687
fourth the seri a prefer stock until all share of seri a prefer stock subject to redempt have been redeem and shall redeem the remain share to have been redeem a soon a practic after the corpor ha fund legal avail therefor thi corpor shall not redeem ani other share of capit stock of the corpor other than secur repurchas from former employe offic director consult or other person who perform servic for the corpor or ani subsidiari in connect with the cessat of such employ or servic at the lower of fair market vaiue or the origin purchas price thereof prior to the redempt of all share of prefer stock pursuant to thi subsect b 6 v except for exclud share a defin below unless otherwis determin by the prior vote or written consent of the holder o f at least a major of the prefer slock on an a convert to common stock basi the redempt price for seri d prefer stock shall mean the greater of a the seri d origin issu price plu all declar but unpaid dividend on a share of

begining 628 end 3199
iv ani other repurchas or redempt of capit stock of the corpor approv by the holder of the common stock and prefer stock of the corpor vote a separ classes  e dividend rate shall mean an annual rate of $0.0714 per share for the seri a prefer stock subject to adjust from time to time for recapit a set forth elsewher herein and an annual rate of $0.0535 per share for the seri a 2 prefer stock subject to adjust from time to time for recapit a set forth elsewher herein   f liquid prefer shall mean $0892 per share for the seri a prefer stock subject to adjust from time to time for recapit a set forth elsewher herein and shall mean $0669 per share for the seri a 2 prefer stock subject to adjust from time to time for recapit a set forth elsewher herein   g option shall mean right option or warrant to subscrib for purchas or otherwis acquir common stock or convert securities  h origin issu price shall mean $0892 per share for the seri a prefer stock subject to adjust from

no begining found
begining 4273 end 4373
be chang to the seri a convers price that would have been in effect at the time of such expir or termin had such option or convert secur never beat issu and the share of common stock issuabl thereund shall no longer be deem to be common stock outstand the number of share of addit share of common stock deem issu and the consider deem paid therefor pursuant to subsect d  3k x i shall be appropri adjust to reflect ani chang termin or expir of the type describ in subsect d  3 b or d  3xc
begining 486 end 771
iv id e n d r ig h t s  a holder of seri b prefer and seri b l prefer pari passuwith each other and in prefer to the holder of seri a prefer and common stock shall be entitl to receiv when a and if declar by the board of director the board but onli out of fund that are legal avail therefor cash dividend at the rate of eight percent 8 of the origin issu price a defin below of the seri b prefer and seri b l prefer respect per annum on each outstan

...

no begining found
begining -60 end 40

begining 5912 end 8004
fourth addit share of common stock shall mean all share of common stock issu or pursuant to subsect 4 4 3 below deem to be issu by the corpor after the seri c origin issu date other than exempt securities  a convert secur shall mean ani evid of indebted share or other secur directli or indirectli convert into or exchang for common stock but exclud options  b the follow share of common stock and 2 share of common stock deem issu pursuant to the follow option and convert secur exempt secur shall mean 1 c 11  share of common orconvert secur issu a a dividend or distribut on prefer stock stock option stock option share of common orconvert secur issu by reason of a dividend stock split split up or other distribut on share of common stock that is cover by subsect 4.5 4 6 4 7 or 48 share of common stock or option issu to employe or director of or consult or advisor to the corpor or ani of it subsidiari pursuant to a plan agreement

no begining found
begining -60 end 40

begining 5912 end 8004
fourth addit share of common stock shall mean all share of common stock issu or pursuant to subsect 4 4 3 below deem to be issu by the corpor after the seri c origin issu date other than exempt securities  a convert secur shall mean ani evid of indebted share or other secur directli or indirectli convert into or exchang for common stock but exclud options  b the follow share of common stock and 2 share of common stock deem issu pursuant to the follow option and convert secur exempt secur shall mean 1 c 11  share of common orconvert secur issu a a dividend or distribut on prefer stock stock option stock option share of common orconvert secur issu by reason of a dividend stock split split up or other distribut on share of common stock that is cover by subsect 4.5 4 6 4 7 or 48 share of common stock or option issu to employe or director of or consult or advisor to the corpor or ani of it subsidiari pursuant to a plan agreement

no begining found
begining -60 end 40

begining 5912 end 8004
fourth addit share of common stock shall mean all share of common stock issu or pursuant to subsect 4 4 3 below deem to be issu by the corpor after the seri c origin issu date other than exempt securities  a convert secur shall mean ani evid of indebted share or other secur directli or indirectli convert into or exchang for common stock but exclud options  b the follow share of common stock and 2 share of common stock deem issu pursuant to the follow option and convert secur exempt secur shall mean 1 c 11  share of common orconvert secur issu a a dividend or distribut on prefer stock stock option stock option share of common orconvert secur issu by reason of a dividend stock split split up or other distribut on share of common stock that is cover by subsect 4.5 4 6 4 7 or 48 share of common stock or option issu to employe or director of or consult or advisor to the corpor or ani of it subsidiari pursuant to a plan agreement

begining 20 end 78
iv is io n o f c o rp o ra tio n s d e liv e r e d 0 5 5 6 pm 0 7 0 3 2 0 1 3 file 0 5 38 pm 0 7 0 3 2 0 1 3 srv 130848560 5283608 filecertif of incorpor ofarvina inc a delawar corporationi
no begining found
begining 869 end 969
of the gener corpor law of california to the extent otherwis applic shall not appli with respect to distribut a hereinaft defin made by the compani in connect with the repurchas of share of common stock issu to or held by employe consult offic or director at a price not greater than the amount paid by such person for such share upon termin of their employ or servic pursuant to agreement provid for the right of said repurchas or upon exercis of a right of first refus which agreement were author by the board of director for purpos of thi
no begining found
begining 969 end 1069
and 1 b whether or not declar ani addit dividend declar shall be distribut among all holder of common stock and prefer stock in proport to the number of share of common s

KeyboardInterrupt: 

In [84]:
BEGINING_OF_IV_INTRO_TEXT = get_proccessed_triggers("Begining_IV")
END_OF_IV_INTRO_TEXT = get_proccessed_triggers("End_IV_intro")
print(BEGINING_OF_IV_INTRO_TEXT)
print(END_OF_IV_INTRO_TEXT)

[['fourth'], ['iv'], ['corpor', 'is', 'author', 'to', 'issu'], ['the', 'total', 'number', 'of', 'share', 'of', 'all', 'class', 'of', 'stock']]
[['the', 'follow', 'is', 'a', 'statement', 'of', 'the', 'design', 'and', 'the', 'right', 'power', 'and', 'prefer'], ['follow', 'is', 'a', 'statement', 'of', 'the', 'design', 'and', 'the', 'right', 'power', 'and', 'prefer'], ['rel', 'right', 'prefer', 'privileg', 'and', 'restrict', 'grant'], ['section', '1', 'dividen'], ['the', 'undersign', 'incorpor'], ['power', 'prefer', 'right', 'restrict', 'and', 'other', 'matter'], ['dividend'], ['amend', 'to', 'restat', 'certif', 'of', 'incorpor', 'ha', 'been', 'duli', 'adopt'], ['fith'], ['the', 'name', 'and', 'mail', 'address', 'of', 'the', 'sole']]


In [1]:
Failures = [
    "/Users/DanielLongo/Dropbox/VC RA Avinika Narayan/Contracts project/coi/Done OCR'd/ActiveSemi International/148_ActiveSemi_COI_11082010.pdf",
]
